<a href="https://colab.research.google.com/github/GreggHeller1/replace_with_env_name/blob/main/scripts/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#settings
%load_ext autoreload
%autoreload 2
try:
  import google.colab
  in_colab = True
except:
  in_colab = False
print(in_colab)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
False


In [10]:
#installs (for colab only, run this once)
if in_colab:
    ! git clone https://github.com/GreggHeller1/replace_with_env_name.git

In [11]:
#local imports
#cwd if in colab for imports to work
if in_colab:
    %cd /content/replace_with_env_name
    
from src import data_io as io
from src import plotting as plot
from src import computation as comp
from src import helper_functions as hf
import main

ModuleNotFoundError: No module named 'xarray'

In [14]:
#imports
import os

#import xarray as xr
#import pandas as pd
#import numpy as np
#import ipdb
#from matplotlib import pyplot as plt
#from PIL import Image #this needs to be after matplotlib??
#from scipy.stats import stats   
#from neuron import h, gui

In [9]:
#define paths
#cwd if in colab for file loading to work
if in_colab:
    %cd /content/replace_with_env_name/scripts
    
print(os.getcwd())
base_dir = os.path.split(os.getcwd())[0]
test_path = os.path.join(base_dir, 'demo_data', 'test.txt')
print(test_path)
print(os.path.exists(test_path))

data_path = "your data path here"
print(os.path.exists(data_path))

demo_data\test.txt
C:\Users\Gregg\Dropbox (MIT)\gregg_code\python-template-repo-surlab\scripts
False
False


In [13]:
#data inputs
loaded_data = main.main_io()

In [10]:
#data manipulation
results = main.main_computation(loaded_data)

In [12]:
#debug speed 
#import cProfile
#cProfile.run('main.main_computation(loaded_data)')

In [ ]:
#save results
main.main_saving(results)

In [ ]:
#reload results for plotting
loaded_results = main.reload_results_for_plotting()

In [ ]:
#make and save plots
main.main_plotting(loaded_results)